In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from selenium import webdriver
from lxml import etree
import os
from os import path
import shutil
import time
import demjson

In [2]:
def getResponse(url):
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36'
    }
    response = requests.get(url=url, headers=headers)
    response.encoding = 'utf-8'
    return response

In [ ]:
# http://movie.medline.org.cn/highInterview.jhtml
# http://movie.medline.org.cn/highInterview/890117.jhtml

In [21]:
# 获取视频下载地址
def getDownloadUrl(url):
    # 需要selenium
    browser = webdriver.Chrome()
    browser.get(url)
    page_text = browser.page_source
    tree = etree.HTML(page_text)
    temp = tree.xpath('//*[@id="media"]/div/div[1]/video/@src')
    if len(temp) == 0:
        return ""
    videoUrl = temp[0]    
    print(videoUrl)
    return videoUrl

In [7]:
videoUrl = getDownloadUrl('http://movie.medline.org.cn/highInterview/890117.jhtml')
print(videoUrl)

http://dpv.videocc.net/35cf77ab8e/7/35cf77ab8e6862106a862bb6cb69c117_2.mp4?pid=1617843886667X1120701
http://dpv.videocc.net/35cf77ab8e/7/35cf77ab8e6862106a862bb6cb69c117_2.mp4?pid=1617843886667X1120701


In [10]:
# 下载视频
def download(url):
    print("开始下载：", url)
    fileName = (url.split('/')[-1]).split('?')[0]
    content = getResponse(url).content
    path = 'video/'
    with open(path + fileName, 'wb') as fp:
        fp.write(content)
    print(fileName, ' download successful')

In [11]:
download('http://dpv.videocc.net/35cf77ab8e/7/35cf77ab8e6862106a862bb6cb69c117_2.mp4?pid=1617843886667X1120701')

开始下载： http://dpv.videocc.net/35cf77ab8e/7/35cf77ab8e6862106a862bb6cb69c117_2.mp4?pid=1617843886667X1120701
35cf77ab8e6862106a862bb6cb69c117_2.mp4  download successful


In [ ]:
# http://movie.medline.org.cn/highInterview_1.jhtml
# http://movie.medline.org.cn/highInterview_10.jhtml

In [12]:
url = 'http://movie.medline.org.cn/highInterview_1.jhtml'
page_text = getResponse(url).text
print(page_text)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<title>高端访谈</title>
<link href="/res/dist/css/jquery.prompt.css" rel="stylesheet" type="text/css"/>
<link href="/r/cms/xinshijie/css/style.css" rel="stylesheet" type="text/css" />
<link href="/r/cms/xinshijie/css/update.css" rel="stylesheet" type="text/css" />
<script src="/r/cms/xinshijie/js/jquery-1.8.2.min.js" type="text/javascript"></script>
<script src="/r/cms/xinshijie/js/jquery.SuperSlide.2.1.1.js" type="text/javascript"></script>
<script src="/r/cms/xinshijie/js/head.js" type="text/javascript"></script>
<script src="/res/dist/js/jquery.prompt.min.js" type="text/javascript"></script>
<script src="/r/cms/front.js" type="text/javascript"></script>
</head>

<body>
	<div class="head">
    	<div class="header">
        	<div class="header_top">


In [16]:
tree = etree.HTML(page_text)
videos = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[4]/div[2]/div[@class="cv_ship clearfix"]')
print(len(videos))
for video in videos:
    videoPageUrl = video.xpath('./div[@class="cv_img"]/a/@href')[0]
    #print(videoPageUrl)

6
http://movie.medline.org.cn:80/highInterview/890117.jhtml
http://movie.medline.org.cn:80/highInterview/890101.jhtml
http://movie.medline.org.cn:80/highInterview/890100.jhtml
http://movie.medline.org.cn:80/highInterview/890099.jhtml
http://movie.medline.org.cn:80/highInterview/889995.jhtml
http://movie.medline.org.cn:80/highInterview/889991.jhtml


In [17]:
# 获取每页的视频地址
def getVideoUrlOfEveryPage(url):
    linkList = []
    page_text = getResponse(url).text
    tree = etree.HTML(page_text)
    videos = tree.xpath('/html/body/div[2]/div[2]/div[2]/div[4]/div[2]/div[@class="cv_ship clearfix"]')
    #print(len(videos))
    for video in videos:
        videoPageUrl = video.xpath('./div[@class="cv_img"]/a/@href')[0]
        #print(videoPageUrl)
        linkList.append(videoPageUrl)
    return linkList

In [18]:
for link in getVideoUrlOfEveryPage('http://movie.medline.org.cn/highInterview_1.jhtml'):
    print(link)

http://movie.medline.org.cn:80/highInterview/890117.jhtml
http://movie.medline.org.cn:80/highInterview/890101.jhtml
http://movie.medline.org.cn:80/highInterview/890100.jhtml
http://movie.medline.org.cn:80/highInterview/890099.jhtml
http://movie.medline.org.cn:80/highInterview/889995.jhtml
http://movie.medline.org.cn:80/highInterview/889991.jhtml


In [28]:
for i in range(8, 11):
    print(f"start download page {i}")
    url = f'http://movie.medline.org.cn/highInterview_{i}.jhtml'
    linkList = getVideoUrlOfEveryPage(url)
    for j, link in enumerate(linkList):
        if i == 8 and j < 1:
            continue
        print(f'page {i} video {j}')
        downloadUrl = getDownloadUrl(link)
        if downloadUrl == "":
            print('error')
            continue
        download(downloadUrl)

start download page 8
page 8 video 1
http://dpv.videocc.net/35cf77ab8e/8/35cf77ab8e93e58178899f911db52c18_2.mp4?pid=1617865097410X1265530
开始下载： http://dpv.videocc.net/35cf77ab8e/8/35cf77ab8e93e58178899f911db52c18_2.mp4?pid=1617865097410X1265530
35cf77ab8e93e58178899f911db52c18_2.mp4  download successful
page 8 video 2
http://dpv.videocc.net/35cf77ab8e/d/35cf77ab8e8eca8d56a9ddf6c9e692dd_2.mp4?pid=1617865136689X1307541
开始下载： http://dpv.videocc.net/35cf77ab8e/d/35cf77ab8e8eca8d56a9ddf6c9e692dd_2.mp4?pid=1617865136689X1307541
35cf77ab8e8eca8d56a9ddf6c9e692dd_2.mp4  download successful
page 8 video 3


TimeoutException: Message: timeout: Timed out receiving message from renderer: 296.713
  (Session info: chrome=89.0.4389.90)
